In [18]:
import os
from langchain import OpenAI
from langchain.vectorstores import Chroma
from langchain.embeddings import OpenAIEmbeddings
from langchain_openai import ChatOpenAI
from langchain.memory import ConversationSummaryBufferMemory
from langchain.utilities import GoogleSearchAPIWrapper
from langchain.retrievers.web_research import WebResearchRetriever
from langchain.chains import RetrievalQAWithSourcesChain

In [5]:
from dotenv import load_dotenv;

# Load Environment variables
load_dotenv()

True

In [9]:
# Vectorstore
vectorstore = Chroma(embedding_function=OpenAIEmbeddings(), persist_directory="./chroma_db_oai")

In [16]:
# LLM
llm = ChatOpenAI(model_name="gpt-3.5-turbo", temperature=0, streaming=True)


In [19]:
# Memory for Retriever
memory = ConversationSummaryBufferMemory(llm=llm, input_key='question', output_key='answer', return_messages=True)

In [21]:
# Search
search = GoogleSearchAPIWrapper()

In [22]:
# Retriever
web_research_retriever = WebResearchRetriever.from_llm(
    vectorstore=vectorstore,
    llm=llm,
    search=search,
)

In [80]:
# Define the User Input
# brand = "CocaCola"
brand = input()

# Initialize question-answering chain with sources retrieval
qa_chain = RetrievalQAWithSourcesChain.from_chain_type(llm, retriever=web_research_retriever)

In [83]:
result_introduction = qa_chain({"question": f"Write an introduction to {brand}"})
print(result_introduction["answer"])
print(result_introduction["sources"])

Fetching pages: 100%|##########| 2/2 [00:00<00:00,  3.02it/s]


Coca-Cola, or Coke, is a carbonated soft drink with a cola flavor manufactured by the Coca-Cola Company. In 2013, Coke products were sold in over 200 countries worldwide, with consumers drinking more than 1.8 billion company beverage servings each day. Coca-Cola ranked No. 87 in the 2018 Fortune 500 list of the largest United States corporations by total revenue. Based on Interbrand's "best global brand" study of 2020, Coca-Cola was the world's sixth most valuable brand. The company has a diverse product portfolio, including carbonated soft drinks, juices, teas, and bottled water, and has a strong focus on corporate social responsibility, sustainability, and community engagement. Coca-Cola has maintained a consistent brand image, emotional branding, storytelling, and a strong global marketing strategy to connect with consumers and maintain brand loyalty.

https://en.wikipedia.org/wiki/Coca-Cola, https://voymedia.com/coca-cola-marketing-strategy/


In [39]:
# Query the QA chain with the user input question
result_history = qa_chain({"question": f"Explain the History of {brand} in detail"})
print(result_history["answer"])
print(result_history["sources"])

The Coca-Cola Company was founded in 1886 in Atlanta, Georgia. Since then, the company now sells more than 1.9 billion beverages in more than 200 countries around the world daily. The company has seen many changes over the years, from using coupons for free drinks to raise interest in the product to being ranked sixth as the world's most valuable brand in 2020. Coca-Cola has managed to retain its popularity and keep up with the times while still remaining entrenched in nostalgia.

https://www.investopedia.com/financial-edge/1012/the-evolution-of-the-coca-cola-brand.aspx


In [40]:
result_demographics = qa_chain({"question": f"Explain the Demographics of {brand} in detail"})
print(result_demographics["answer"])
print(result_demographics["sources"])

Fetching pages: 100%|##########| 3/3 [00:01<00:00,  1.59it/s]


Coca-Cola's demographic segmentation includes a broad age range of younger to mid-life consumers, from 10 to 40 years old. The geographic segmentation of Coca-Cola's target audience is located all over the world. The behavioral segmentation shows that the target audience consumes the product regularly. The psychographic segmentation of Coca-Cola's target audience includes a wide socio-economic breakdown. The company targets young people between 10 and 35, middle-aged and older adults, and individuals based on gender. Coca-Cola sells its products globally and targets different cultures, customs, and climates. The largest age group in terms of Coca-Cola consumption is ages 35-44.

https://www.start.io/blog/coca-cola-target-market-segmentation-customer-analysis-marketing-strategy/, https://www.simplilearn.com/tutorials/marketing-case-studies-tutorial/coca-cola-marketing-strategy


In [43]:
result_social_presence = qa_chain({"question": f"Explain the Social presence of {brand} in detail"})
print(result_social_presence["answer"])
print(result_social_presence["sources"])

Fetching pages: 100%|##########| 2/2 [00:03<00:00,  1.95s/it]


Coca-Cola has a strong social media presence across various platforms including Facebook, Twitter, Instagram, and YouTube. They actively engage with their audience, respond to inquiries, create interactive experiences, and run promotional campaigns. Coca-Cola also prioritizes customer engagement, community building, real-time marketing, and user-generated content to strengthen brand loyalty and advocacy. Their social media strategy focuses on creating engaging content, interacting with the audience, and supporting social causes, showcasing a commitment to more than just profit. 

https://keyhole.co/blog/coca-cola-social-media-strategy/, https://voymedia.com/coca-cola-marketing-strategy/


In [47]:
result_market_activity  = qa_chain({"question": f"Explain the Market Activity  of {brand}"})
print(result_market_activity["answer"])
print(result_market_activity["sources"])

Coca-Cola was the top carbonated soft drink company in the US in 2020, with a volume share of 44.9%. Their marketing strategy focuses on selling a lifestyle promise rather than just a product, and they use a vast, multi-channel approach. Coca-Cola also engages in real-time marketing on social media platforms and prioritizes customer engagement and community building. They have a diverse product portfolio and emphasize corporate social responsibility in their marketing strategy.

https://www.start.io/blog/coca-cola-target-market-segmentation-customer-analysis-marketing-strategy/, https://voymedia.com/coca-cola-marketing-strategy/, https://keyhole.co/blog/coca-cola-social-media-strategy/, https://www.investopedia.com/financial-edge/1012/the-evolution-of-the-coca-cola-brand.aspx


In [87]:
# Create a Markdown file Article

from mdutils.mdutils import MdUtils
from mdutils import Html
md = MdUtils(file_name=f"{brand}")
md.new_header(level=1, title=f"{brand}")
md.new_paragraph(f'{result_introduction["answer"]}\n')
md.new_header(level=2, title="History")
md.new_paragraph(f'{result_history["answer"]}\n')
md.new_header(level=2, title="Demographics")
md.new_paragraph(f'{result_demographics["answer"]}\n')
md.new_header(level=2, title="Social presence")
md.new_paragraph(f'{result_social_presence["answer"]}\n')
md.new_header(level=2, title="Market activity")
md.new_paragraph(f'{result_market_activity["answer"]}\n')
md.new_header(level=2, title="Reference Articles")
references = result_introduction["sources"].split(',')
references.extend(result_history["sources"].split(','))
references.extend(result_demographics["sources"].split(','))
references.extend(result_social_presence["sources"].split(','))
references.extend(result_market_activity["sources"].split(','))
md.new_list(references)
md.create_md_file()

In [4]:
from md2pdf.core import md2pdf

markdown_file = f"output/{brand}.md"
output_pdf = f"output/{brand}.pdf"

# md2pdf(output_pdf,
#        md=markdown_file,
#        raw=None,
#        css="md.css",
#        base_url=None,
#        extras=[],
# )
md2pdf(pdf_file_path=output_pdf, css_file_path="md.css", md_file_path=markdown_file)
